<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/Enhanced_optimization_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data

In [1]:
try:
    import mip
except ImportError:
    import sys
    !{sys.executable} -m pip install mip
import pandas as pd
import numpy as np
import re
import random
from mip import Model, BINARY, CONTINUOUS, xsum, maximize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.6/462.6 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: cffi
    Found existing installation: cffi 1.17.1
    Uninstalling cffi-1.17.1:
      Successfully uninstalled cffi-1.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pygit2 1.18.0 requires cffi>=1.17.0, but you have cffi 1.15.1 which is incompatible.


# Define league settings

In [2]:
year = 2024
num_rounds = 15
num_teams = 12
num_weeks = 17
allowed_positions = {'QB', 'RB', 'WR', 'TE', 'K', 'DST'}
lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'FLEX': 1, 'K': 1, 'DST': 1}

## Load player ADP, projections and bye weeks

In [3]:
bye_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/bye_weeks/bye_weeks_{year}.csv'
bye_weeks = pd.read_csv(bye_url)
bye_weeks

,Bye,Team,Abbreviation
0,Week_5,Lions,DET
1,Week_5,Chargers,LAC
2,Week_5,Eagles,PHI
3,Week_5,Titans,TEN
4,Week_6,Chiefs,KC
5,Week_6,Rams,LA
6,Week_6,Dolphins,MIA
7,Week_6,Vikings,MIN
8,Week_7,Bears,CHI
9,Week_7,Cowboys,DAL


In [4]:
adp_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/pre_season_data/adp_projections_{year}.csv'

player_adp = (
    pd.read_csv(adp_url)
      .rename(columns={'player': 'Player', 'position': 'POS', 'adp': 'ADP', 'points': 'TTL'})
      .loc[:, ['Player', 'POS', 'ADP', 'TTL']]
      .sort_values('ADP')
      .reset_index(drop=True)
)

print(player_adp.value_counts('POS'))
player_adp.head(15)


POS
WR     72
RB     72
QB     36
TE     36
K      32
DST    32
Name: count, dtype: int64


,Player,POS,ADP,TTL
0,Christian McCaffrey,RB,1.36,353.0
1,CeeDee Lamb,WR,3.88,348.0
2,Tyreek Hill,WR,4.02,334.0
3,Breece Hall,RB,4.98,309.0
4,Bijan Robinson,RB,5.19,303.0
5,Amon-Ra St. Brown,WR,7.50,313.0
6,Ja'Marr Chase,WR,7.92,307.0
7,Justin Jefferson,WR,8.64,308.0
8,Jonathan Taylor,RB,10.20,269.0
9,Saquon Barkley,RB,10.60,276.0


In [5]:
season_projection_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/main/pre_season_data/adp_projections_{year}.csv'

season_projections = (
    pd.read_csv(season_projection_url)
      .rename(columns={'player': 'Player', 'position': 'POS', 'team': 'Team', 'points': 'TTL', 'adp': 'ADP'})
      .loc[:, ['Player', 'POS', 'Team', 'TTL', 'ADP']]
      .merge(bye_weeks[['Abbreviation', 'Bye']], left_on='Team', right_on='Abbreviation', how='left')
      .drop(columns='Abbreviation')
)
season_projections = season_projections.drop_duplicates(subset='Player', keep='first')
season_projections

,Player,POS,Team,TTL,ADP,Bye
0,Christian McCaffrey,RB,SF,353.0,1.36,Week_9
1,Breece Hall,RB,NYJ,309.0,4.98,Week_12
2,Bijan Robinson,RB,ATL,303.0,5.19,Week_12
3,CeeDee Lamb,WR,DAL,348.0,3.88,Week_7
4,Tyreek Hill,WR,MIA,334.0,4.02,Week_6
...,...,...,...,...,...,...
274,Emari Demercado,RB,ARI,56.9,170.00,Week_11
275,Drake Maye,QB,NE,153.0,136.00,Week_14
276,Jacoby Brissett,QB,NE,108.0,170.00,Week_14
277,Justin Fields,QB,PIT,107.0,176.00,Week_9


In [6]:
season_projections['avg_proj'] = season_projections['TTL'] / np.where(
    season_projections['Bye'].isna(),
    num_weeks,
    num_weeks - 1
)

week_cols = [f'Week_{w}' for w in range(1, num_weeks + 1)]
for week in week_cols:
    season_projections[week] = np.where(
        season_projections['Bye'] == week,
        0,                          # Bye-Week bekommt 0
        season_projections['avg_proj']  # alle anderen Wochen bekommen avg_proj
    )

# 5. Aufräumen (optional)
season_projections = season_projections.drop(columns='avg_proj')
print(season_projections)

                  Player POS Team    TTL     ADP      Bye     Week_1  \
0    Christian McCaffrey  RB   SF  353.0    1.36   Week_9  22.062500   
1            Breece Hall  RB  NYJ  309.0    4.98  Week_12  19.312500   
2         Bijan Robinson  RB  ATL  303.0    5.19  Week_12  18.937500   
3            CeeDee Lamb  WR  DAL  348.0    3.88   Week_7  21.750000   
4            Tyreek Hill  WR  MIA  334.0    4.02   Week_6  20.875000   
..                   ...  ..  ...    ...     ...      ...        ...   
274      Emari Demercado  RB  ARI   56.9  170.00  Week_11   3.556250   
275           Drake Maye  QB   NE  153.0  136.00  Week_14   9.562500   
276      Jacoby Brissett  QB   NE  108.0  170.00  Week_14   6.750000   
277        Justin Fields  QB  PIT  107.0  176.00   Week_9   6.687500   
278      Aidan O'Connell  QB  LVR   68.4  170.00      NaN   4.023529   

        Week_2     Week_3     Week_4  ...     Week_8     Week_9    Week_10  \
0    22.062500  22.062500  22.062500  ...  22.062500   0.

In [34]:
weekly_projections_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/projection_data/2024/weekly_projections_{year}.csv'

weekly_projections = (
    pd.read_csv(weekly_projections_url)
      .loc[:, ['player', 'position', 'team', 'points', 'week', 'year']]
      .rename(columns={'player': 'Player', 'position': 'POS', 'Team': 'NFL_Team'})
      .query('POS in @allowed_positions')
      .copy()
)

weekly_projections = (
    weekly_projections
      .pivot_table(
         index=['Player','POS'],
         columns='week',
         values='points',
         aggfunc='first'
      )
      .rename_axis(None, axis=1)
      .add_prefix('Week_')
      .reset_index()
      .fillna(0)
)
weekly_projections

,Player,POS,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,Week_7,Week_8,Week_9,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17,Week_18
0,49ers,DST,5.42,5.95,5.15,5.59,5.39,5.40,4.58,5.64,0.00,5.40,5.83,4.54,3.89,5.59,4.54,4.67,4.11,4.32
1,A.J. Brown,WR,17.20,17.80,0.00,0.00,0.00,15.40,16.10,16.30,17.50,15.50,15.40,16.50,18.00,16.00,14.60,15.60,15.30,0.00
2,AJ Barner,TE,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.30,0.00,5.60,4.07,0.00,0.00,0.00,0.00,0.00,0.00
3,Aaron Jones,RB,12.30,12.40,12.40,14.60,14.90,0.00,14.00,15.30,17.00,15.70,14.30,13.70,14.60,13.90,14.10,13.90,14.40,14.10
4,Aaron Rodgers,QB,13.70,15.10,14.10,15.80,14.10,13.40,15.50,15.80,14.10,16.30,15.60,0.00,14.00,13.20,15.90,15.50,15.00,13.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,Zach Ertz,TE,6.49,7.11,6.84,8.48,7.50,7.51,8.12,7.80,9.01,8.57,8.20,9.34,9.79,0.00,9.74,8.24,8.59,9.05
473,Zack Moss,RB,10.10,11.20,13.60,13.70,11.70,11.10,9.18,8.99,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
474,Zamir White,RB,10.90,9.71,10.70,8.42,0.00,4.74,6.48,3.92,3.27,0.00,4.22,3.80,0.00,0.00,0.00,0.00,0.00,0.00
475,Zane Gonzalez,K,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,8.74,8.69,1.50,9.35,0.00,9.04,9.21,7.48,7.37


## create custom projections

In [8]:
# f = weekly_projections.pivot_table(
#     index=['Player', 'POS'],
#     columns='week',
#     values='points',
#     aggfunc='first'
# )
f = season_projections

# Rename columns to 'Week X'
# f.columns = [f'Week_{col}' for col in f.columns]
# f['TTL'] = f.sum(axis=1)
# f = f.reset_index()
# f = f.fillna(0)
# f = f.merge(player_adp[['Player', 'ADP']], on='Player', how='left')

# f = f.drop_duplicates(subset='Player', keep='first')

# calculate dropoff inside grouped POS
f['dropoff'] = (f.sort_values(['POS','TTL'], ascending=[True, False]).groupby('POS')['TTL'].diff(-1).fillna(0.0))
dropoff_weight = {'QB': 1.0, 'RB': 1.0, 'WR': 1.0, 'TE': 0.9, 'K': 0.4, 'DST': 0.3}
f['dropoff'] = f.apply(lambda row: row['dropoff'] * dropoff_weight.get(row['POS'], 1.0), axis=1)

# calculate value-over-replacement (VOR)
vor_dict = {}
vor_weight = {'QB': 0.8, 'RB': 1.0, 'WR': 1.0, 'TE': 0.8, 'K': 0.25, 'DST': 0.25}
for pos, limit in lineup_req.items():
    replacement_index = limit * num_teams - 1
    pos_df = f[f['POS'] == pos].sort_values('TTL', ascending=False).reset_index(drop=True)
    if len(pos_df) > replacement_index:
        replacement_ttl = pos_df.loc[replacement_index, 'TTL']
    else:
        replacement_ttl = 0

    weight = vor_weight.get(pos, 1.0)

    for idx, row in pos_df.iterrows():
        vor = (row['TTL'] - replacement_ttl) * weight
        vor_dict[row['Player']] = row['TTL'] - replacement_ttl

f = f.sort_values(['ADP', 'TTL'],  ascending=[True, False]).reset_index(drop=True)
f['VOR'] = f['Player'].map(vor_dict)
head_size = 300
print(len(f))
f.head(head_size)

279


,Player,POS,Team,TTL,ADP,Bye,Week_1,Week_2,Week_3,Week_4,...,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17,dropoff,VOR
0,Christian McCaffrey,RB,SF,353.0,1.36,Week_9,22.062500,22.062500,22.062500,22.062500,...,22.062500,22.062500,22.062500,22.062500,22.062500,22.062500,22.062500,22.062500,44.00,162.0
1,CeeDee Lamb,WR,DAL,348.0,3.88,Week_7,21.750000,21.750000,21.750000,21.750000,...,21.750000,21.750000,21.750000,21.750000,21.750000,21.750000,21.750000,21.750000,14.00,119.0
2,Tyreek Hill,WR,MIA,334.0,4.02,Week_6,20.875000,20.875000,20.875000,20.875000,...,20.875000,20.875000,20.875000,20.875000,20.875000,20.875000,20.875000,20.875000,21.00,105.0
3,Breece Hall,RB,NYJ,309.0,4.98,Week_12,19.312500,19.312500,19.312500,19.312500,...,19.312500,19.312500,0.000000,19.312500,19.312500,19.312500,19.312500,19.312500,6.00,118.0
4,Bijan Robinson,RB,ATL,303.0,5.19,Week_12,18.937500,18.937500,18.937500,18.937500,...,18.937500,18.937500,0.000000,18.937500,18.937500,18.937500,18.937500,18.937500,27.00,112.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,Giants,DST,NYG,85.5,185.00,Week_11,5.343750,5.343750,5.343750,5.343750,...,5.343750,0.000000,5.343750,5.343750,5.343750,5.343750,5.343750,5.343750,0.06,-5.9
275,John Stephens Jr.,WR,DAL,145.0,NaN,Week_7,9.062500,9.062500,9.062500,9.062500,...,9.062500,9.062500,9.062500,9.062500,9.062500,9.062500,9.062500,9.062500,0.00,-84.0
276,Darren Waller,TE,FA,106.0,NaN,NaN,6.235294,6.235294,6.235294,6.235294,...,6.235294,6.235294,6.235294,6.235294,6.235294,6.235294,6.235294,6.235294,4.50,-50.0
277,Kareem Hunt,RB,FA,103.0,NaN,NaN,6.058824,6.058824,6.058824,6.058824,...,6.058824,6.058824,6.058824,6.058824,6.058824,6.058824,6.058824,6.058824,1.00,-88.0


In [9]:
# positions = ['QB', 'RB', 'WR', 'TE', 'K', 'DST']
# sampled_players = []

# for pos in positions:
#     top_50 = f[f['POS'] == pos] \
#         .sort_values('TTL', ascending=False) \
#         .head(20)

#     if not top_50.empty:
#         sampled_players.append(top_50.sample(1))

# # Ergebnisse kombinieren
# sample_df = pd.concat(sampled_players).reset_index(drop=True)

# # Spaltenauswahl (optional)
# sample_df = sample_df[['Player', 'POS', 'Week_1', 'Week_2', 'Week_17', 'TTL', 'dropoff', 'VOR']]

# print(sample_df)

In [10]:
duplicates = f[f.duplicated(subset=['Player'], keep=False)]

if duplicates.empty:
    print("No duplicates existing")
else:
    print(duplicates.sort_values('Player'))

No duplicates existing


## Create Player dataset

In [11]:
players = f[['Player', 'POS', 'ADP', 'TTL']].copy()
players = players.rename(columns={
    'Player': 'Player',
    'POS': 'POS',
    'TTL': 'TTL'
})
players = players.sort_values(['ADP', 'TTL'], ascending=[True, False]).reset_index(drop=True)
players['Rank'] = players.index + 1

players.info()
print(len(players))
print(players)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 279 entries, 0 to 278
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  279 non-null    object 
 1   POS     279 non-null    object 
 2   ADP     275 non-null    float64
 3   TTL     279 non-null    float64
 4   Rank    279 non-null    int64  
dtypes: float64(2), int64(1), object(2)
memory usage: 11.0+ KB
279
                  Player  POS     ADP    TTL  Rank
0    Christian McCaffrey   RB    1.36  353.0     1
1            CeeDee Lamb   WR    3.88  348.0     2
2            Tyreek Hill   WR    4.02  334.0     3
3            Breece Hall   RB    4.98  309.0     4
4         Bijan Robinson   RB    5.19  303.0     5
..                   ...  ...     ...    ...   ...
274               Giants  DST  185.00   85.5   275
275    John Stephens Jr.   WR     NaN  145.0   276
276        Darren Waller   TE     NaN  106.0   277
277          Kareem Hunt   RB     NaN  103.0   278
278      

### discarded

# Initiate Model and apply Optimization

In [12]:
# # ==========================
# # 1. PARAMETERS
# # ==========================

# players_list = players['Player'].copy().tolist()
# positions = {'QB', 'RB', 'WR', 'TE', 'K', 'DST'}
# weeks = list(range(1, 18))
# pos = dict(zip(weekly_projections['Player'], weekly_projections['POS']))
# lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}
# week_cols = [col for col in f.columns if col.startswith('Week_')]

# # Build f_dict: weekly projections and dropoff for each player
# f_dict = {
#     row['Player']: {
#         **{int(week.replace('Week_', '')): row[week] for week in week_cols},
#         'dropoff': row['dropoff'],
#         'VOR': row['VOR']
#     }
#     for _, row in f.iterrows()
# }

# beta = {t: 120.0 for t in weeks}
# min_req = {'QB': 2, 'RB': 4, 'WR': 4, 'TE': 2, 'K': 1, 'DST': 1}
# max_req = {'QB': 3, 'RB': 8, 'WR': 8, 'TE': 3, 'K': 1, 'DST': 1}
# alpha, lambda_0, lambda_1, lambda_2, lambda_3, lambda_4 = 1.0, 1, 3, 4, 0.5, 400
# df_sorted = players.sort_values('Rank').reset_index(drop=True)
# topk_pct = 0.0025

NameError: name 'weekly_projections' is not defined

In [ ]:
# # ==========================
# # 2. INITIALIZATION (TEAMS, DRAFT ORDER, OPPONENT PICK)
# # ==========================

# teams = [f'Team {i+1}' for i in range(num_teams)]
# DM_team = 'Team 1'

# # Snake draft order
# draft_order = []
# for rnd in range(num_rounds):
#     order = teams if rnd % 2 == 0 else teams[::-1]
#     draft_order += order

# def opponent_pick(roster, available, Rk, lineup_req, topk_pct=topk_pct):
#     # 1) Sort remaining players by rank
#     remaining_players = sorted(available, key=lambda p: Rk[p])
#     topk = max(1, int(len(remaining_players) * topk_pct))

#     # 2) Calculate deficits per position (min requirement minus current roster)
#     deficits = {
#         j: lineup_req[j] - sum(1 for p in roster if pos[p] == j)
#         for j in lineup_req
#     }
#     needed = [j for j, d in deficits.items() if d > 0]

#     # 3) If deficits exist, pick from candidates in those positions
#     if needed:
#         candidates = [p for p in remaining_players if pos[p] in needed]
#         pool = candidates[:topk] if len(candidates) >= topk else candidates
#         if pool:
#             return random.choice(pool)

#     # 4) Fallback: pick randomly from top-k overall
#     return random.choice(remaining_players[:topk])

# # ==========================
# # 3. DRAFT INITIALIZATION
# # ==========================

# rosters = {tm: [] for tm in teams}
# available = set(players_list)
# draft_log = []

# # ==========================
# # 4. MAIN DRAFT LOOP
# # ==========================

# for pick_idx, team in enumerate(draft_order, start=1):

#     # ---- 4.1: Update remaining player ranking ----
#     for p in available:
#       ranks = df_sorted.loc[df_sorted.Player == p, 'Rank']
#       if len(ranks) != 1:
#           print(f'Problem bei Spieler {p}: Anzahl gefundener Ränge = {len(ranks)}')

#     remaining_players = sorted(available, key=lambda p: df_sorted.loc[df_sorted.Player == p, 'Rank'].item())
#     Rk = {p: i+1 for i, p in enumerate(remaining_players)}
#     player_vars = set(remaining_players) | set(rosters[team])
#     picks_remaining = num_rounds - len(rosters[team])

#     # ---- 4.2: DM-Team (your team) picks via MIP ----
#     if team == DM_team:
#         m = Model(sense=maximize, solver_name='CBC')

#         # --- Decision variables ---
#         # y[i]: 1 if player i is drafted by DM, 0 otherwise
#         y = {i: m.add_var(var_type=BINARY, name=f'y_{i}') for i in player_vars}
#         # x[i, t]: share of player i's points used in week t (continuous)
#         x = {(i, t): m.add_var(var_type=CONTINUOUS, name=f'x_{i}_{t}') for i in remaining_players for t in weeks}
#         # z[t]: 1 if DM wins in week t, 0 otherwise
#         z = {t: m.add_var(var_type=BINARY, name=f'z_{t}') for t in weeks}

#         # --- Objective function ---
#         m.objective = (
#             lambda_0 * xsum(f_dict[i][t] * x[i, t] for i in remaining_players for t in weeks)
#             + lambda_1 * xsum(z[t] for t in weeks[:15])
#             + lambda_2 * xsum(z[t] for t in weeks[15:])
#             + lambda_3 * xsum(f_dict[i]['dropoff'] * y[i] for i in remaining_players)
#             - lambda_4 * xsum(f_dict[i]['VOR'] * y[i] for i in remaining_players)
#         )

#         # --- Constraints ---
#         # Fix previous picks (already drafted players must remain picked)
#         for p in rosters[team]:
#             m += y[p] == 1

#         # Enforce that exactly the remaining picks are made
#         m += xsum(y[i] for i in remaining_players) == picks_remaining

#         # Enforce minimum requirements for each position (relative to already drafted players)
#         for pos_name, req in lineup_req.items():
#             already_satisfied = sum(1 for p in rosters[team] if pos[p] == pos_name)
#             need = max(0, req - already_satisfied)
#             m += xsum(y[i] for i in remaining_players if pos[i] == pos_name) >= need

#         # Weekly lineup constraints and position limits
#         for j in positions:
#             # m += xsum(y[i] for i in remaining_players if pos[i] == j) >= gamma[j], f'{j}_min_req'  # Minimum number per position
#             for t in weeks:
#                 m += xsum(x[i, t] for i in remaining_players if pos[i] == j) <= lineup_req[j]  # Weekly lineup limit

#         # Only drafted players can be in the weekly lineup
#         for i in remaining_players:
#             for t in weeks:
#                 m += x[i, t] <= y[i]

#         # Win indicator constraints
#         for t in weeks:
#             m += z[t] <= xsum(f_dict[i][t] * x[i, t] for i in remaining_players) / beta[t]

#         # Robust draft constraint (to simulate uncertainty in opponent picks)
#         n_k = pick_idx
#         for future_pick in range(
#             pick_idx + 1,
#             pick_idx + picks_remaining * len(teams),
#             len(teams)
#         ):
#             top_cut = int(alpha * (future_pick - n_k))
#             if top_cut > 0:
#                 top_players = [i for i, r in Rk.items() if r <= top_cut]
#                 m += xsum(y[i] for i in top_players) <= ((future_pick - n_k) // len(teams))

#         # --- Solve the MIP model ---
#         m.optimize()
#         # Check for infeasibility
#         if m.num_solutions == 0:
#             raise RuntimeError(f'No feasible solution at pick {pick_idx}. Check constraints and player pool.')

#         print(f"\n--- DM Pick {pick_idx} ---")
#         for i in remaining_players:
#             if y[i].x is not None and y[i].x >= 0.9:
#                 ttl = sum(f_dict[i][t] for t in weeks)
#                 vor = vor_dict.get(i, 0.0)
#                 print(f"{i:25} ({pos[i]})  TTL={ttl:6.1f}   VOR={vor:6.1f}")


#         # Extract chosen player for this pick
#         chosen = [i for i in remaining_players if y[i].x is not None and y[i].x >= 0.99 and i not in rosters[team]]
#         if not chosen:
#             raise RuntimeError(f'No feasible pick at {pick_idx}')
#         pick = min(chosen, key=lambda i: Rk[i])

#     # ---- 4.3: Opponent pick (simple heuristic) ----
#     else:
#         current_round = (pick_idx - 1) // len(teams) + 1
#         dynamic_topk_pct = min(current_round * 2 * topk_pct, 1.0)
#         pick = opponent_pick(
#             roster=rosters[team],
#             available=available,
#             Rk=Rk,
#             lineup_req=lineup_req,
#             topk_pct=dynamic_topk_pct
#         )

#     # ---- 4.4: Update rosters and draft log ----
#     rosters[team].append(pick)
#     available.remove(pick)
#     draft_log.append({
#         'Pick': pick_idx,
#         'Team': team,
#         'Player': pick,
#         'Round': (pick_idx - 1) // len(teams) + 1,
#         'POS': pos[pick]
#     })

# # ==========================
# # 5. CREATE DRAFT DATAFRAME
# # ==========================

# df_draft = pd.DataFrame(draft_log)


In [ ]:
# # mit flex:
# # ==========================
# # 1. PARAMETERS
# # ==========================

# players_list = players['Player'].copy().tolist()
# positions = {'QB', 'RB', 'WR', 'TE', 'K', 'DST', 'FLEX'}
# flex_eligible = {'RB', 'WR', 'TE'}
# weeks = list(range(1, 18))
# pos = dict(zip(weekly_projections['Player'], weekly_projections['POS']))
# lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1, 'FLEX': 1}
# week_cols = [col for col in f.columns if col.startswith('Week_')]

# # Build f_dict: weekly projections and dropoff for each player
# f_dict = {
#     row['Player']: {
#         **{int(week.replace('Week_', '')): row[week] for week in week_cols},
#         'dropoff': row['dropoff'],
#         'VOR': row['VOR']
#     }
#     for _, row in f.iterrows()
# }

# beta = {t: 100.0 for t in weeks}
# gamma = {'QB': 2, 'RB': 3, 'WR': 3, 'TE': 2, 'K': 1, 'DST': 1}
# alpha, lambda_0, lambda_1, lambda_2, lambda_3, lambda_4 = 1.0, 1, 3, 4, 0.5, 400
# df_sorted = players.sort_values('Rank').reset_index(drop=True)
# topk_pct = 0.0025

# # ==========================
# # 2. INITIALIZATION (TEAMS, DRAFT ORDER, OPPONENT PICK)
# # ==========================

# teams = [f'Team {i+1}' for i in range(num_teams)]
# DM_team = 'Team 1'

# # Snake draft order
# draft_order = []
# for rnd in range(num_rounds):
#     order = teams if rnd % 2 == 0 else teams[::-1]
#     draft_order += order

# def opponent_pick(roster, available, Rk, lineup_req, topk_pct=topk_pct):
#     remaining_players = sorted(available, key=lambda p: Rk[p])
#     topk = max(1, int(len(remaining_players) * topk_pct))

#     deficits = {
#         j: lineup_req[j] - sum(1 for p in roster if pos[p] == j)
#         for j in lineup_req if j != 'FLEX'
#     }
#     needed = [j for j, d in deficits.items() if d > 0]

#     if needed:
#         candidates = [p for p in remaining_players if pos[p] in needed]
#         pool = candidates[:topk] if len(candidates) >= topk else candidates
#         if pool:
#             return random.choice(pool)

#     return random.choice(remaining_players[:topk])

# # ==========================
# # 3. DRAFT INITIALIZATION
# # ==========================

# rosters = {tm: [] for tm in teams}
# available = set(players_list)
# draft_log = []

# # ==========================
# # 4. MAIN DRAFT LOOP
# # ==========================

# for pick_idx, team in enumerate(draft_order, start=1):

#     # ---- 4.1: Update remaining player ranking ----
#     for p in available:
#         ranks = df_sorted.loc[df_sorted.Player == p, 'Rank']
#         if len(ranks) != 1:
#             print(f'Problem bei Spieler {p}: Anzahl gefundener Ränge = {len(ranks)}')

#     remaining_players = sorted(available, key=lambda p: df_sorted.loc[df_sorted.Player == p, 'Rank'].item())
#     Rk = {p: i+1 for i, p in enumerate(remaining_players)}
#     player_vars = set(remaining_players) | set(rosters[team])
#     picks_remaining = num_rounds - len(rosters[team])

#     # ---- 4.2: DM-Team (your team) picks via MIP ----
#     if team == DM_team:
#         m = Model(sense=maximize, solver_name='CBC')

#         # --- Decision variables ---
#         y = {i: m.add_var(var_type=BINARY, name=f'y_{i}') for i in player_vars}
#         x = {(i, t): m.add_var(var_type=CONTINUOUS, name=f'x_{i}_{t}') for i in remaining_players for t in weeks}
#         z = {t: m.add_var(var_type=BINARY, name=f'z_{t}') for t in weeks}

#         # --- Objective function ---
#         m.objective = (
#             lambda_0 * xsum(f_dict[i][t] * x[i, t] for i in remaining_players for t in weeks)
#             + lambda_1 * xsum(z[t] for t in weeks[:15])
#             + lambda_2 * xsum(z[t] for t in weeks[15:])
#             + lambda_3 * xsum(f_dict[i]['dropoff'] * y[i] for i in remaining_players)
#             - lambda_4 * xsum(f_dict[i]['VOR'] * y[i] for i in remaining_players)
#         )

#         # --- Constraints ---
#         for p in rosters[team]:
#             m += y[p] == 1

#         m += xsum(y[i] for i in remaining_players) == picks_remaining

#         for pos_name, req in lineup_req.items():
#             if pos_name != 'FLEX':
#                 already_satisfied = sum(1 for p in rosters[team] if pos[p] == pos_name)
#                 need = max(0, req - already_satisfied)
#                 m += xsum(y[i] for i in remaining_players if pos[i] == pos_name) >= need

#         for t in weeks:
#             # Standard-Positionen (ohne FLEX)
#             for j in positions - {'FLEX'}:
#                 m += xsum(x[i, t] for i in remaining_players if pos[i] == j) <= lineup_req.get(j, 0)
#             # FLEX-Position: Summe der eligible Spieler
#             m += xsum(x[i, t] for i in remaining_players if pos[i] in flex_eligible) <= lineup_req['FLEX']

#             # Kein Spieler darf in einer Woche mehr als einen Slot belegen (Stamm + FLEX)
#             for i in remaining_players:
#                 m += (
#                     (x[i, t] if pos[i] in positions - {'FLEX'} else 0) +
#                     (x[i, t] if pos[i] in flex_eligible else 0)
#                     <= 1
#                 )

#         for i in remaining_players:
#             for t in weeks:
#                 m += x[i, t] <= y[i]

#         for t in weeks:
#             m += z[t] <= xsum(f_dict[i][t] * x[i, t] for i in remaining_players) / beta[t]

#         n_k = pick_idx
#         for future_pick in range(
#             pick_idx + 1,
#             pick_idx + picks_remaining * len(teams),
#             len(teams)
#         ):
#             top_cut = int(alpha * (future_pick - n_k))
#             if top_cut > 0:
#                 top_players = [i for i, r in Rk.items() if r <= top_cut]
#                 m += xsum(y[i] for i in top_players) <= ((future_pick - n_k) // len(teams))

#         # --- Solve the MIP model ---
#         m.optimize()
#         if m.num_solutions == 0:
#             raise RuntimeError(f'No feasible solution at pick {pick_idx}. Check constraints and player pool.')

#         print(f"\n--- DM Pick {pick_idx} ---")
#         for i in remaining_players:
#             if y[i].x is not None and y[i].x >= 0.9:
#                 ttl = sum(f_dict[i][t] for t in weeks)
#                 vor = vor_dict.get(i, 0.0)
#                 print(f"{i:25} ({pos[i]})  TTL={ttl:6.1f}   VOR={vor:6.1f}")

#         chosen = [i for i in remaining_players if y[i].x is not None and y[i].x >= 0.99 and i not in rosters[team]]
#         if not chosen:
#             raise RuntimeError(f'No feasible pick at {pick_idx}')
#         pick = min(chosen, key=lambda i: Rk[i])

#     # ---- 4.3: Opponent pick (simple heuristic) ----
#     else:
#         current_round = (pick_idx - 1) // len(teams) + 1
#         dynamic_topk_pct = min(current_round * 2 * topk_pct, 1.0)
#         pick = opponent_pick(
#             roster=rosters[team],
#             available=available,
#             Rk=Rk,
#             lineup_req=lineup_req,
#             topk_pct=dynamic_topk_pct
#         )

#     # ---- 4.4: Update rosters and draft log ----
#     rosters[team].append(pick)
#     available.remove(pick)
#     draft_log.append({
#         'Pick': pick_idx,
#         'Team': team,
#         'Player': pick,
#         'Round': (pick_idx - 1) // len(teams) + 1,
#         'POS': pos[pick]
#     })

# # ==========================
# # 5. CREATE DRAFT DATAFRAME
# # ==========================

# df_draft = pd.DataFrame(draft_log)


In [13]:
# mit flex:
# ==========================
# 1. PARAMETERS
# ==========================

players_list = players['Player'].copy().tolist()
positions = {'QB', 'RB', 'WR', 'TE', 'K', 'DST', 'FLEX'}
flex_eligible = {'RB', 'WR', 'TE'}
weeks = list(range(1, 18))
pos = dict(zip(season_projections['Player'], season_projections['POS']))
lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1, 'FLEX': 1}
max_req = {'QB': 3, 'RB': 999, 'WR': 999, 'TE': 2, 'K': 1, 'DST': 1}
week_cols = [col for col in f.columns if col.startswith('Week_')]

# Build f_dict: weekly projections and dropoff for each player
f_dict = {
    row['Player']: {
        **{int(week.replace('Week_', '')): row[week] for week in week_cols},
        'dropoff': row['dropoff'],
        'VOR': row['VOR']
    }
    for _, row in f.iterrows()
}

beta = {t: 120.0 for t in weeks}
alpha, lambda_0, lambda_1, lambda_2, lambda_3, lambda_4 = 1.0, 1, 3, 4, 0.5, 4
df_sorted = players.sort_values('Rank').reset_index(drop=True)
topk_pct = 0.0025

# ==========================
# 2. INITIALIZATION (TEAMS, DRAFT ORDER, OPPONENT PICK)
# ==========================

teams = [f'Team {i+1}' for i in range(num_teams)]
DM_team = 'Team 1'

# Snake draft order
draft_order = []
for rnd in range(num_rounds):
    order = teams if rnd % 2 == 0 else teams[::-1]
    draft_order += order

def opponent_pick(roster, available, Rk, lineup_req, topk_pct=topk_pct):
    remaining_players = sorted(available, key=lambda p: Rk[p])
    topk = max(1, int(len(remaining_players) * topk_pct))

    deficits = {
        j: lineup_req[j] - sum(1 for p in roster if pos[p] == j)
        for j in lineup_req if j != 'FLEX'
    }
    needed = [j for j, d in deficits.items() if d > 0]

    if needed:
        candidates = [p for p in remaining_players if pos[p] in needed]
        pool = candidates[:topk] if len(candidates) >= topk else candidates
        if pool:
            return random.choice(pool)

    return random.choice(remaining_players[:topk])

# ==========================
# 3. DRAFT INITIALIZATION
# ==========================

rosters = {tm: [] for tm in teams}
available = set(players_list)
draft_log = []

# ==========================
# 4. MAIN DRAFT LOOP
# ==========================

for pick_idx, team in enumerate(draft_order, start=1):

    # ---- 4.1: Update remaining player ranking ----
    for p in available:
        ranks = df_sorted.loc[df_sorted.Player == p, 'Rank']
        if len(ranks) != 1:
            print(f'Problem bei Spieler {p}: Anzahl gefundener Ränge = {len(ranks)}')

    remaining_players = sorted(available, key=lambda p: df_sorted.loc[df_sorted.Player == p, 'Rank'].item())
    Rk = {p: i+1 for i, p in enumerate(remaining_players)}
    player_vars = set(remaining_players) | set(rosters[team])
    picks_remaining = num_rounds - len(rosters[team])

    # ---- 4.2: DM-Team (your team) picks via MIP ----
    if team == DM_team:
        m = Model(sense=maximize, solver_name='CBC')

        # --- Decision variables ---
        y = {i: m.add_var(var_type=BINARY, name=f'y_{i}') for i in player_vars}
        x = {(i, t): m.add_var(var_type=CONTINUOUS, name=f'x_{i}_{t}') for i in remaining_players for t in weeks}
        z = {t: m.add_var(var_type=BINARY, name=f'z_{t}') for t in weeks}

        # --- Objective function ---
        m.objective = (
            lambda_0 * xsum(f_dict[i][t] * x[i, t] for i in remaining_players for t in weeks)
            + lambda_1 * xsum(z[t] for t in weeks[:15])
            + lambda_2 * xsum(z[t] for t in weeks[15:])
            + lambda_3 * xsum(f_dict[i]['dropoff'] * y[i] for i in remaining_players)
            - lambda_4 * xsum(f_dict[i]['VOR'] * y[i] for i in remaining_players)
        )

        # --- Constraints ---
        for p in rosters[team]:
            m += y[p] == 1

        m += xsum(y[i] for i in remaining_players) == picks_remaining

        for pos_name, req in lineup_req.items():
            if pos_name != 'FLEX':
                already_satisfied = sum(1 for p in rosters[team] if pos[p] == pos_name)
                need = max(0, req - already_satisfied)
                m += xsum(y[i] for i in remaining_players if pos[i] == pos_name) >= need

        for t in weeks:
            # Standard-Positionen (ohne FLEX)
            for j in positions - {'FLEX'}:
                # mindestens
                # m += xsum(y[i] for i in remaining_players if pos[i] == j) >= lineup_req[j]
                # höchstens
                m += xsum(y[i] for i in remaining_players if pos[i] == j) <= max_req[j]

            # Kein Spieler darf in einer Woche mehr als einen Slot belegen (Stamm + FLEX)
            for i in remaining_players:
                m += (
                    (x[i, t] if pos[i] in positions - {'FLEX'} else 0) +
                    (x[i, t] if pos[i] in flex_eligible else 0)
                    <= 1
                )

        for i in remaining_players:
            for t in weeks:
                m += x[i, t] <= y[i]

        for t in weeks:
            m += z[t] <= xsum(f_dict[i][t] * x[i, t] for i in remaining_players) / beta[t]

        n_k = pick_idx
        for future_pick in range(
            pick_idx + 1,
            pick_idx + picks_remaining * len(teams),
            len(teams)
        ):
            top_cut = int(alpha * (future_pick - n_k))
            if top_cut > 0:
                top_players = [i for i, r in Rk.items() if r <= top_cut]
                m += xsum(y[i] for i in top_players) <= ((future_pick - n_k) // len(teams))

        # --- Solve the MIP model ---
        m.optimize()
        if m.num_solutions == 0:
            raise RuntimeError(f'No feasible solution at pick {pick_idx}. Check constraints and player pool.')

        print(f"\n--- DM Pick {pick_idx} ---")
        for i in remaining_players:
            if y[i].x is not None and y[i].x >= 0.9:
                ttl = sum(f_dict[i][t] for t in weeks)
                vor = vor_dict.get(i, 0.0)
                print(f"{i:25} ({pos[i]})  TTL={ttl:6.1f}   VOR={vor:6.1f}")

        chosen = [i for i in remaining_players if y[i].x is not None and y[i].x >= 0.99 and i not in rosters[team]]
        if not chosen:
            raise RuntimeError(f'No feasible pick at {pick_idx}')
        pick = min(chosen, key=lambda i: Rk[i])

    # ---- 4.3: Opponent pick (simple heuristic) ----
    else:
        current_round = (pick_idx - 1) // len(teams) + 1
        dynamic_topk_pct = min(current_round * 2 * topk_pct, 1.0)
        pick = opponent_pick(
            roster=rosters[team],
            available=available,
            Rk=Rk,
            lineup_req=lineup_req,
            topk_pct=dynamic_topk_pct
        )

    # ---- 4.4: Update rosters and draft log ----
    rosters[team].append(pick)
    available.remove(pick)
    draft_log.append({
        'Pick': pick_idx,
        'Team': team,
        'Player': pick,
        'Round': (pick_idx - 1) // len(teams) + 1,
        'POS': pos[pick]
    })

# ==========================
# 5. CREATE DRAFT DATAFRAME
# ==========================

df_draft = pd.DataFrame(draft_log)



--- DM Pick 1 ---
CeeDee Lamb               (WR)  TTL= 348.0   VOR= 119.0
Kyren Williams            (RB)  TTL= 254.0   VOR=  63.0
Josh Jacobs               (RB)  TTL= 243.0   VOR=  52.0
Alvin Kamara              (RB)  TTL= 241.0   VOR=  50.0
Trey McBride              (TE)  TTL= 208.0   VOR=  52.0
Rhamondre Stevenson       (RB)  TTL= 205.0   VOR=  14.0
Javonte Williams          (RB)  TTL= 191.0   VOR=   0.0
Devin Singletary          (RB)  TTL= 186.0   VOR=  -5.0
Caleb Williams            (QB)  TTL= 292.0   VOR=   0.0
Dallas Goedert            (TE)  TTL= 163.0   VOR=   7.0
Trevor Lawrence           (QB)  TTL= 291.0   VOR=  -1.0
Jakobi Meyers             (WR)  TTL= 182.0   VOR= -47.0
Younghoe Koo              (K)  TTL= 156.0   VOR=  12.0
Deshaun Watson            (QB)  TTL= 273.0   VOR= -19.0
Colts                     (DST)  TTL=  95.8   VOR=   4.4

--- DM Pick 24 ---
Josh Jacobs               (RB)  TTL= 243.0   VOR=  52.0
Alvin Kamara              (RB)  TTL= 241.0   VOR=  50.0
Aaron Jon

In [14]:
print(m.status)
#print(m.num_constrs, m.num_vars)
print(m)

OptimizationStatus.OPTIMAL


## Inspect results

In [15]:
merged = pd.merge(df_draft, f[['Player', 'TTL']], on='Player', how='left')

# Replace missing TTLs (e.g., für Spieler ohne Projection) mit 0
merged['TTL'] = merged['TTL'].fillna(0)

# Group by Team and sum TTL to get total projection per team
team_ttl_proj = merged.groupby('Team')['TTL'].sum().reset_index()
team_ttl_proj = team_ttl_proj.rename(columns={'TTL': 'TTL_proj'})
print(team_ttl_proj)

position_counts_df = df_draft.groupby(['Team', 'POS']).size().unstack(fill_value=0)
print(position_counts_df)

result_dfs = {}

for team in df_draft['Team'].unique():
    team_df = df_draft[df_draft['Team'] == team].copy()
    team_df = team_df.sort_values(by='Pick')

    # Optional: Spalten anpassen, wenn nur bestimmte Infos gewünscht sind
    team_df["Pick Info"] = team_df.apply(lambda row: f"Round {row['Round']} Pick {row['Pick']}", axis=1)
    result_dfs[team] = team_df[['Player', 'Pick Info', 'POS']]

for team, df in result_dfs.items():
    print(f'=== {team} ===')
    print(df)
    print()

       Team  TTL_proj
0    Team 1    3382.8
1   Team 10    2959.0
2   Team 11    3099.3
3   Team 12    2735.6
4    Team 2    3204.6
5    Team 3    2880.3
6    Team 4    2921.7
7    Team 5    2869.1
8    Team 6    3146.0
9    Team 7    3220.8
10   Team 8    2868.2
11   Team 9    2936.5
POS      DST  K  QB  RB  TE  WR
Team                           
Team 1     1  1   3   5   3   2
Team 10    1  1   1   4   1   7
Team 11    1  1   2   3   1   7
Team 12    1  1   2   5   2   4
Team 2     1  1   3   4   1   5
Team 3     1  1   1   5   2   5
Team 4     2  1   2   6   1   3
Team 5     1  1   1   4   2   6
Team 6     1  2   3   4   1   4
Team 7     1  1   3   2   1   7
Team 8     2  1   1   5   1   5
Team 9     1  2   1   5   1   5
=== Team 1 ===
               Player          Pick Info  POS
0         CeeDee Lamb     Round 1 Pick 1   WR
23        Josh Jacobs    Round 2 Pick 24   RB
24      Rachaad White    Round 3 Pick 25   RB
47       Alvin Kamara    Round 4 Pick 48   RB
48       Mark Andrews

# Evaluation

## merge df_draft with f

In [37]:
roster_projections = (
    df_draft
      .merge(
         weekly_projections.drop(columns=['POS', 'Week_18']),
         on='Player',
         how='left'
      )
      .sort_values(['Team','Pick'])
      .sort_values(by=['Team', 'Pick'])
      .reset_index(drop=True)
)
roster_projections = roster_projections
roster_projections

,Pick,Team,Player,Round,POS,Week_1,Week_2,Week_3,Week_4,Week_5,...,Week_8,Week_9,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17
0,1,Team 1,CeeDee Lamb,1,WR,18.30,20.80,20.70,20.20,19.80,...,17.50,21.00,16.60,15.10,15.80,16.70,16.70,15.60,16.80,0.00
1,24,Team 1,Josh Jacobs,2,RB,14.30,13.80,14.90,14.10,13.10,...,13.80,14.00,0.00,14.00,15.80,16.80,16.40,17.50,19.70,18.60
2,25,Team 1,Rachaad White,3,RB,16.00,13.60,13.50,12.00,11.10,...,11.50,10.90,11.30,0.00,12.40,12.00,11.00,12.30,12.00,12.70
3,48,Team 1,Alvin Kamara,4,RB,15.50,14.80,19.20,17.90,17.60,...,15.70,19.50,19.90,20.20,0.00,19.60,21.90,16.50,0.00,5.28
4,49,Team 1,Mark Andrews,5,TE,11.70,10.70,10.50,8.77,6.72,...,8.14,8.75,9.65,9.48,8.61,9.02,0.00,10.50,9.23,9.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,129,Team 9,Calvin Ridley,11,WR,12.00,11.10,11.90,11.30,0.00,...,10.40,12.00,11.20,12.10,12.20,12.80,13.00,13.10,12.30,12.80
177,136,Team 9,Blake Corum,12,RB,5.90,0.00,2.31,0.00,0.00,...,2.89,3.27,2.57,2.22,2.57,2.84,3.09,3.34,3.65,3.63
178,153,Team 9,Tyler Allgeier,13,RB,6.72,6.00,6.44,6.34,7.13,...,7.39,7.78,7.40,6.35,0.00,6.19,5.32,5.73,5.90,5.41
179,160,Team 9,Romeo Doubs,14,WR,9.56,7.88,7.95,9.10,0.00,...,11.20,10.10,0.00,9.87,9.33,0.00,0.00,9.09,9.83,9.65


# create schedule

In [38]:
num_reg_weeks = 14
playoff_weeks = [15, 16, 17]

def create_reg_schedule(teams):
    n = len(teams)
    schedule = []
    for week in range(num_reg_weeks):
        week_matches = []
        for i in range(n//2):
            team1 = teams[i]
            team2 = teams[n-1-i]
            week_matches.append((team1, team2))
        schedule.append(week_matches)
        # Rotate teams except the first one
        teams = [teams[0]] + [teams[-1]] + teams[1:-1]
    return schedule

reg_schedule = create_reg_schedule(teams)
reg_schedule

[[('Team 1', 'Team 12'),
  ('Team 2', 'Team 11'),
  ('Team 3', 'Team 10'),
  ('Team 4', 'Team 9'),
  ('Team 5', 'Team 8'),
  ('Team 6', 'Team 7')],
 [('Team 1', 'Team 11'),
  ('Team 12', 'Team 10'),
  ('Team 2', 'Team 9'),
  ('Team 3', 'Team 8'),
  ('Team 4', 'Team 7'),
  ('Team 5', 'Team 6')],
 [('Team 1', 'Team 10'),
  ('Team 11', 'Team 9'),
  ('Team 12', 'Team 8'),
  ('Team 2', 'Team 7'),
  ('Team 3', 'Team 6'),
  ('Team 4', 'Team 5')],
 [('Team 1', 'Team 9'),
  ('Team 10', 'Team 8'),
  ('Team 11', 'Team 7'),
  ('Team 12', 'Team 6'),
  ('Team 2', 'Team 5'),
  ('Team 3', 'Team 4')],
 [('Team 1', 'Team 8'),
  ('Team 9', 'Team 7'),
  ('Team 10', 'Team 6'),
  ('Team 11', 'Team 5'),
  ('Team 12', 'Team 4'),
  ('Team 2', 'Team 3')],
 [('Team 1', 'Team 7'),
  ('Team 8', 'Team 6'),
  ('Team 9', 'Team 5'),
  ('Team 10', 'Team 4'),
  ('Team 11', 'Team 3'),
  ('Team 12', 'Team 2')],
 [('Team 1', 'Team 6'),
  ('Team 7', 'Team 5'),
  ('Team 8', 'Team 4'),
  ('Team 9', 'Team 3'),
  ('Team 10', 'T

## Load real life player data

In [39]:
data_url = f'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/Weekly_Data/weekly_data_{year}.csv'
weekly_data = pd.read_csv(data_url)
weekly_data

,player_display_name,position,season,week,fantasy_points_ppr
0,Aaron Rodgers,QB,2024,1,8.58
1,Aaron Rodgers,QB,2024,2,15.14
2,Aaron Rodgers,QB,2024,3,21.04
3,Aaron Rodgers,QB,2024,4,11.60
4,Aaron Rodgers,QB,2024,5,11.76
...,...,...,...,...,...
5592,Trey Benson,RB,2024,10,10.70
5593,Trey Benson,RB,2024,12,1.80
5594,Trey Benson,RB,2024,13,2.00
5595,Trey Benson,RB,2024,14,2.90


In [45]:
# Roster limits
lineup_req = {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'K': 1, 'DST': 1}

# Für jede Woche und jedes Team die beste Aufstellung bestimmen
def get_best_lineup(team, week, roster_projections, lineup_req):
    week_col = f'Week_{week}'
    team_roster = roster_projections[roster_projections['Team'] == team]
    lineup = []
    used_players = set()

    for pos, limit in lineup_req.items():
        if pos != 'FLEX':
            candidates = team_roster[team_roster['POS'] == pos]
        else:
            # FLEX-Kandidaten: RB, WR, TE, die noch nicht verwendet wurden
            candidates = team_roster[
                (team_roster['POS'].isin(['RB', 'WR', 'TE'])) &
                (~team_roster['Player'].isin(used_players))
            ]

        starters = candidates.sort_values(week_col, ascending=False).head(limit)
        lineup.append(starters)
        used_players.update(starters['Player'])

    return pd.concat(lineup)


# Punkte aus weekly_data holen
def get_actual_points(lineup, week, weekly_data):
    merged = lineup.merge(
        weekly_data[weekly_data['week'] == week],
        left_on='Player', right_on='player_display_name', how='left'
    )
    # Fülle fehlende Werte (z.B. bei Bye Weeks) mit 0
    merged['fantasy_points_ppr'] = merged['fantasy_points_ppr'].fillna(0)
    return merged['fantasy_points_ppr'].sum()

# Für alle Wochen und alle Matchups durchlaufen
results = []
for week_idx, matchups in enumerate(reg_schedule, 1):
    for team1, team2 in matchups:
        lineup1 = get_best_lineup(team1, week_idx, roster_projections, lineup_req)
        lineup2 = get_best_lineup(team2, week_idx, roster_projections, lineup_req)
        points1 = get_actual_points(lineup1, week_idx, weekly_data)
        points2 = get_actual_points(lineup2, week_idx, weekly_data)
        winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
        loser = team1 if points1 < points2 else team2 if points2 < points1 else 'Unentschieden'
        results.append({
            'Woche': week_idx,
            'Team 1': team1,
            'Team 2': team2,
            'Punkte Team 1': points1,
            'Punkte Team 2': points2,
            'Sieger': winner,
            'Verlierer': loser
        })
df_results = pd.DataFrame(results)
df_results

,Woche,Team 1,Team 2,Punkte Team 1,Punkte Team 2,Sieger,Verlierer
0,1,Team 1,Team 12,82.82,87.72,Team 12,Team 1
1,1,Team 2,Team 11,68.84,102.46,Team 11,Team 2
2,1,Team 3,Team 10,79.92,90.02,Team 10,Team 3
3,1,Team 4,Team 9,87.26,79.16,Team 4,Team 9
4,1,Team 5,Team 8,76.78,55.70,Team 5,Team 8
...,...,...,...,...,...,...,...
79,14,Team 11,Team 9,53.06,72.30,Team 9,Team 11
80,14,Team 12,Team 8,66.60,78.04,Team 8,Team 12
81,14,Team 2,Team 7,96.40,102.40,Team 7,Team 2
82,14,Team 3,Team 6,113.92,133.88,Team 6,Team 3


In [48]:
def get_lineup_and_bench(team, week, roster_projections, weekly_data, lineup_req):
    week_col = f'Week_{week}'
    team_roster = roster_projections[roster_projections['Team'] == team].copy()

    # Bestes Lineup bestimmen
    lineup = get_best_lineup(team, week, roster_projections, lineup_req)

    # Bench = Spieler aus Roster, die NICHT im Lineup sind
    lineup_players = set(lineup['Player'])
    bench = team_roster[~team_roster['Player'].isin(lineup_players)].copy()

    # Füge tatsächliche Punkte hinzu
    week_actuals = weekly_data[weekly_data['week'] == week][['player_display_name', 'fantasy_points_ppr']]
    week_actuals = week_actuals.rename(columns={'player_display_name': 'Player', 'fantasy_points_ppr': 'Actual_Points'})

    for df in [lineup, bench]:
        df['Projected_Points'] = df[week_col]
        df.drop(columns=[col for col in df.columns if col.startswith("Week_")], inplace=True)
        df = df.merge(week_actuals, on='Player', how='left')
        df['Actual_Points'] = df['Actual_Points'].fillna(0)

    return lineup, bench
lineup, bench = get_lineup_and_bench(DM_team, 1, roster_projections, weekly_data, lineup_req)

import pandas as pd
pd.set_option('display.max_rows', 30)  # falls du viele Spieler siehst

print("=== Lineup (Week 1) ===")
display(lineup[['Player', 'POS', 'Projected_Points', 'Actual_Points']])

print("=== Bench (Week 1) ===")
display(bench[['Player', 'POS', 'Projected_Points', 'Actual_Points']])


=== Lineup (Week 1) ===


KeyError: "['Actual_Points'] not in index"

In [46]:
# Deine vorhandene Liste mit Teams
teams = [f'Team {i+1}' for i in range(num_teams)]

# Sieger und Verlierer zählen (Unentschieden ausschließen)
wins = df_results[df_results['Sieger'] != 'Unentschieden']['Sieger'].value_counts()
losses = df_results[df_results['Verlierer'] != 'Unentschieden']['Verlierer'].value_counts()

# Draws zählen: alle Teams, die in einem Unentschieden beteiligt waren
draws = (
    df_results[df_results['Sieger'] == 'Unentschieden'][['Team 1', 'Team 2']]
    .stack()
    .value_counts()
)

points_for = pd.concat([
    df_results[['Team 1', 'Punkte Team 1']].rename(columns={'Team 1': 'Team', 'Punkte Team 1': 'Points'}),
    df_results[['Team 2', 'Punkte Team 2']].rename(columns={'Team 2': 'Team', 'Punkte Team 2': 'Points'})
])
points_for = points_for.groupby('Team')['Points'].sum()

# Punkte, die jedes Team kassiert hat ("Points Against")
points_against = pd.concat([
    df_results[['Team 1', 'Punkte Team 2']].rename(columns={'Team 1': 'Team', 'Punkte Team 2': 'Points'}),
    df_results[['Team 2', 'Punkte Team 1']].rename(columns={'Team 2': 'Team', 'Punkte Team 1': 'Points'})
])
points_against = points_against.groupby('Team')['Points'].sum()

# Zusammenführen in ein DataFrame
record = pd.DataFrame({'Team': teams})
record['Wins'] = record['Team'].map(wins).fillna(0).astype(int)
record['Losses'] = record['Team'].map(losses).fillna(0).astype(int)
record['Draws'] = record['Team'].map(draws).fillna(0).astype(int)
record['Points For'] = record['Team'].map(points_for).fillna(0)
record['Points Against'] = record['Team'].map(points_against).fillna(0)
record = record.sort_values(by=['Wins', 'Draws', 'Points For'], ascending=[False, False, False]).reset_index(drop=True)

record

,Team,Wins,Losses,Draws,Points For,Points Against
0,Team 6,9,5,0,1264.70,1137.42
1,Team 12,9,5,0,1257.30,1104.04
2,Team 11,8,6,0,1311.30,1208.72
3,Team 3,8,6,0,1191.88,1178.02
4,Team 10,8,6,0,1149.42,1091.74
5,Team 9,8,6,0,1113.56,1147.20
6,Team 5,7,7,0,1140.74,1219.26
7,Team 7,7,7,0,1099.98,1255.86
8,Team 4,7,7,0,1045.92,1018.14
9,Team 2,6,8,0,1133.36,1145.28


In [47]:
# Setze die Playoff-Wochen
playoff_weeks = [15, 16, 17]

# Teams nach Rang sortieren (wie zuvor)
ranked_teams = record['Team'].tolist()

# Woche 15: Seed 3 vs 6, Seed 4 vs 5
week_15_matchups = [
    (ranked_teams[2], ranked_teams[5]),  # Match 1
    (ranked_teams[3], ranked_teams[4])   # Match 2
]

# Ergebnisse Woche 15
week15_results = []
for team1, team2 in week_15_matchups:
    lineup1 = get_best_lineup(team1, 15, roster_projections, lineup_req)
    lineup2 = get_best_lineup(team2, 15, roster_projections, lineup_req)
    points1 = get_actual_points(lineup1, 15, weekly_data)
    points2 = get_actual_points(lineup2, 15, weekly_data)
    winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
    loser = team1 if points1 < points2 else team2 if points2 < points1 else 'Unentschieden'
    week15_results.append({
        'Woche': 15,
        'Team 1': team1,
        'Team 2': team2,
        'Punkte Team 1': points1,
        'Punkte Team 2': points2,
        'Sieger': winner,
        'Verlierer': loser
    })

# Extrahiere Gewinner
winners_15 = [r['Sieger'] for r in week15_results]

# Woche 16 Matchups:
# Match 3: Winner Match 1 vs Seed 2
# Match 4: Winner Match 2 vs Seed 1
week_16_matchups = [
    (winners_15[0], ranked_teams[1]),  # gegen Seed 2
    (winners_15[1], ranked_teams[0])   # gegen Seed 1
]

week16_results = []
for team1, team2 in week_16_matchups:
    lineup1 = get_best_lineup(team1, 16, roster_projections, lineup_req)
    lineup2 = get_best_lineup(team2, 16, roster_projections, lineup_req)
    points1 = get_actual_points(lineup1, 16, weekly_data)
    points2 = get_actual_points(lineup2, 16, weekly_data)
    winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
    loser = team1 if points1 < points2 else team2 if points2 < points1 else 'Unentschieden'
    week16_results.append({
        'Woche': 16,
        'Team 1': team1,
        'Team 2': team2,
        'Punkte Team 1': points1,
        'Punkte Team 2': points2,
        'Sieger': winner,
        'Verlierer': loser
    })

# Extrahiere Gewinner
winners_16 = [r['Sieger'] for r in week16_results]

# Woche 17: Finale
week17_matchups = [(winners_16[0], winners_16[1])]

week17_results = []
for team1, team2 in week17_matchups:
    lineup1 = get_best_lineup(team1, 17, roster_projections, lineup_req)
    lineup2 = get_best_lineup(team2, 17, roster_projections, lineup_req)
    points1 = get_actual_points(lineup1, 17, weekly_data)
    points2 = get_actual_points(lineup2, 17, weekly_data)
    winner = team1 if points1 > points2 else team2 if points2 > points1 else 'Unentschieden'
    loser = team1 if points1 < points2 else team2 if points2 < points1 else 'Unentschieden'
    week17_results.append({
        'Woche': 17,
        'Team 1': team1,
        'Team 2': team2,
        'Punkte Team 1': points1,
        'Punkte Team 2': points2,
        'Sieger': winner,
        'Verlierer': loser
    })

# Ergebnisse zusammenführen
df_playoff_results = pd.DataFrame(week15_results + week16_results + week17_results)

# Optional an bestehende Ergebnisse anhängen:
df_results = pd.concat([df_results, df_playoff_results], ignore_index=True)

# Finale anzeigen
champion = df_playoff_results[df_playoff_results['Woche'] == 17]['Sieger'].values[0]
print(f"🏆 Der Champion ist: {champion}")
df_playoff_results

🏆 Der Champion ist: Team 9


,Woche,Team 1,Team 2,Punkte Team 1,Punkte Team 2,Sieger,Verlierer
0,15,Team 11,Team 9,68.44,101.64,Team 9,Team 11
1,15,Team 3,Team 10,106.00,56.74,Team 3,Team 10
2,16,Team 9,Team 12,131.12,104.78,Team 9,Team 12
3,16,Team 3,Team 6,75.44,65.16,Team 3,Team 6
4,17,Team 9,Team 3,120.18,75.80,Team 9,Team 3
